# Integration Testing

There are total 56 test cases for the integration testing:
- [28 test cases for float type, double type, float array, double array and the operations with the same types](#link1)
- [9 test cases for type conversins](#link2)
- [4 test cases for unicode symbols](#link3)
- [3 test cases for operators of max and min](#link4)
- [6 test cases for quantified expressions](#link5)
- [6 test cases for programs with procedures using float or double parameters](#link6)

In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString

def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            writeInt: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeDouble: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeFloat: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            readInt: () => window.prompt(),
            readDouble: () => window.prompt(),
            readFloat: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))

Importing Jupyter notebook from P0.ipynb
Importing Jupyter notebook from SC.ipynb
Importing Jupyter notebook from ST.ipynb


In [2]:
def runpywasm(wasmfile):
    import pywasm
    def writeInt(s, i): print(i)
    def writeFloat(s, i): print(i)
    def writeDouble(s, i): print(i)
    def writeln(s): print('\n')
    def readInt(s): return int(input())
    def readDouble(s): return float(input())
    def readFloat(s): return float(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'writeInt': writeInt, 'writeFloat': writeFloat, 'writeDouble': writeDouble, \
                                          'writeln': writeln, 'readInt': readInt, 'readFloat': readFloat, 'readDouble': readDouble}})

<a name="link1"></a>
## 1 Float type and double type

### 1.1 Read and write floats and doubles

#####  Write float type
```should output 
30.858511
3.4028235e+38
````

In [3]:
compileString("""
program writeDouble
    const x = 30.85851114555f
    const y = 340282350000000000000000000000000000000.11111111111111111111f
    write(x)    
    write(y)
""", "writeFloat1.wat")
!wat2wasm writeFloat1.wat
runpywasm("writeFloat1.wasm")

Importing Jupyter notebook from CGwat.ipynb
30.858511
3.4028235e+38


#####  Write double type
```should output 
5.666767677866768
1.797693e+308
```

In [4]:
compileString("""
program writeDouble
    const x = 5.666767677866768755555
    write(x)
    write(179769300000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000.1)
""", "writeDouble1.wat")
!wat2wasm writeDouble1.wat
runpywasm("writeDouble1.wasm")

5.666767677866768
1.797693e+308


#####  Read float type
`should output the number(float type, with at most 7 decimal points) you enter in the box`

In [5]:
compileString("""
var x: float
program readFloat
    x ← read()
    write(x)
""", "read.wat")
!wat2wasm read.wat
runpywasm("read.wasm")

5
5.0


#####  Read double type
`should output the number(float type, with at most 15 decimal points) you enter in the box`

In [6]:
compileString("""
var x: double
program readDouble
    x ← read()
    write(x)
""", "read.wat")
!wat2wasm read.wat
runpywasm("read.wasm")

5.666666
5.666666


### 1.2 Array of float and double types

 #####  Array of double type
 ```
 should output 
 6.666767677866768
 5.666767677866768
 ```

In [7]:
compileString("""
type T = [1..10] → double
var a: T
program assignArrayDouble
  var x, y: double
    a[3] := 5.666767677866768755555
    x, y := a[3], 6.666767677866768755555
    x, y := y, x
    write(x); write(y)
""", 'assignArrayDouble.wat', target = 'wat')
!wat2wasm assignArrayDouble.wat
runpywasm('assignArrayDouble.wasm')

6.666767677866768
5.666767677866768


  #####  Array of float type
 ```
 should output
 6.6667676
 5.6667676
 ```

In [8]:
compileString("""
type T = [1..10] → float
var a: T
program assignArrayFloat
  var x, y: float
    a[3] := 5.666767677866768755555F
    x, y := a[3], 6.666767677866768755555f
    x, y := y, x
    write(x); write(y)
""", 'assignArrayFloat.wat', target = 'wat')
!wat2wasm assignArrayFloat.wat
runpywasm('assignArrayFloat.wasm')

6.6667676
5.6667676


### 1.3 Binary and relation operations for the same type

#####  Binary operation for float types
```
should output
10.6
-5.6
20.25
0.30864197
```

In [9]:
compileString("""
program binaryOpFloat
    write(2.5f + 8.1f)
    write(2.5f - 8.1f)
    write(2.5f × 8.1f)
    write(2.5f div 8.1f)
""", "binaryOpFloat.wat")
!wat2wasm binaryOpFloat.wat
runpywasm("binaryOpFloat.wasm")

10.6
-5.6
20.25
0.30864197


#####  Binary operation for double types
```
should output
10.6
-5.6
20.25
0.308641975308642
```

In [10]:
compileString("""
program binaryOpDouble
    write(2.5 + 8.1)
    write(2.5 - 8.1)
    write(2.5 × 8.1)
    write(2.5 div 8.1)
""", "binaryOpDouble.wat")
!wat2wasm binaryOpDouble.wat
runpywasm("binaryOpDouble.wasm")

10.6
-5.6
20.25
0.308641975308642


#####  Relation operations for double types
```
should output
0 (false)
1 (true)
0 (false)
1 (true)
0 (false)
1 (true)
```

In [11]:
compileString("""
program relationdDouble
    var x, y: double
      x := 2.1; y := 2.13;
      if x = y then write(1) else write(0)
      if x ≠ y then write(1) else write(0)      
      if x > y then write(1) else write(0)
      if x < y then write(1) else write(0)      
      if x ≥ y then write(1) else write(0)
      if x ≤ y then write(1) else write(0)
""", "relationdDouble.wat")
!wat2wasm relationdDouble.wat
runpywasm("relationdDouble.wasm")

0
1
0
1
0
1


#### Multiplication testing for Float and Double type

TIMES operation for float type output 20.25

In [12]:
compileString("""
var x: float
var y: float
var r: float
program p
    x := 8.1
    y := 2.5
    r := x × y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

20.25


TIMES operation for double type output 20.25

In [13]:
compileString("""
var x: double
var y: double
var r: double
program p
    x := 8.1
    y := 2.5
    r := x × y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

20.25


TIMES operation for float and double returns double output 20.25

In [14]:
compileString("""
var x: float
var y: double
var r: double
program p
    x := 8.1f
    y := 2.5
    r := x × y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

20.250000953674316


Times operation for float and integer returns float output 16.2

In [15]:
compileString("""
var x: float
var y: integer
var r: float
program p
    x := 8.1f
    y := 2
    r := x × y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

16.2


Times operation for double and integer returns double output 20.0

In [16]:
compileString("""
var x: integer
var y: double
var r: double
program p
    x := 8
    y := 2.5
    r := x × y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

20.0


#### Division testing for float and double type

DIV operation for float type output 3.24

In [17]:
compileString("""
var x: float
var y: float
var r: float
program p
    x := 8.1f
    y := 2.5f
    r := x div y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

3.2400002


DIV operation for double type output 3.24

In [18]:
compileString("""
var x: double
var y: double
var r: double
program p
    x := 8.1
    y := 2.5
    r := x div y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

3.2399999999999998


DIV operation for float and double type returns double (higher precision). output 3.24

In [19]:
compileString("""
var x: float
var y: double
var r: double
program p
    x := 8.1f
    y := 2.5
    r := x div y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

3.2400001525878905


DIV operation for float and integer returns float (higher precision). output 4.05

In [20]:
compileString("""
var x: float
var y: integer
var r: float
program p
    x := 8.1f
    y := 2
    r := x div y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

4.05


DIV operation for double and integer returns double (higher precision). output 4.05

In [26]:
compileString("""
var x: double
var y: integer
var r: double
program p
    x := 8.1
    y := 2
    r := x div y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

4.05


#### Addition testing for float and double types

PLUS operation for float type output 10.6

In [27]:
compileString("""
var x: float
var y: float
var r: float
program p
    x := 8.1f
    y := 2.5f
    r := x + y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

10.6


PLUS operation for double type output 10.6

In [28]:
compileString("""
var x: double
var y: double
var r: double
program p
    x := 8.1
    y := 2.5
    r := x + y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

10.6


PLUS operation for float and double returns double (higher precision). output 10.6

In [29]:
compileString("""
var x: float
var y: double
var r: double
program p
    x := 8.1f
    y := 2.5
    r := x + y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

10.600000381469727


PLUS operation for float and integer returns float (higher precision). output 10.1

In [30]:
compileString("""
var x: float
var y: integer
var r: float
program p
    x := 8.1f
    y := 2
    r := x + y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

10.1


PLUS operation for double and integer returns double (higher precision). output 10.1

In [31]:
compileString("""
var x: double
var y: integer
var r: double
program p
    x := 8.1
    y := 2
    r := x + y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

10.1


#### Subtraction testing for both float and double types

MINUS operation for float type output 5.6

In [32]:
compileString("""
var x: float
var y: float
var r: float
program p
    x := 8.1f
    y := 2.5f
    r := x - y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

5.6000004


MINUS operation for double type output 5.6

In [33]:
compileString("""
var x: double
var y: double
var r: double
program p
    x := 8.1
    y := 2.5
    r := x - y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

5.6


MINUS operation for float and double returns double (higher precision). output 5.6

In [34]:
compileString("""
var x: float
var y: double
var r: double
program p
    x := 8.1f
    y := 2.5
    r := x - y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

5.600000381469727


MINUS operation for float and integer returns float (higher precision). output 6.1

In [35]:
compileString("""
var x: float
var y: integer
var r: float
program p
    x := 8.1f
    y := 2
    r := x - y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

6.1000004


MINUS operation for double and integer returns double (higher precision). output 5.5

In [36]:
compileString("""
var x: integer
var y: double
var r: double
program p
    x := 8
    y := 2.5
    r := x - y
    write(r)
""", "arith.wat")
!wat2wasm arith.wat
runpywasm("arith.wasm")

5.5


<a name="link2"></a>
## 2 Type conversion

### 2.1 Implicit type conversion

#####   Assign smaller type to larger type
```
should output
5.0
5.0
5.777777194976807
```

In [37]:
compileString("""
var x: float
var y: double
program p
    x := 5            //assign integer to Float type variable
    y := 5           //assign integer to Double type variable    
    write(x)
    write(y)
    y := 5.777777f   //assign float to Double type variable 
    write(y)
""", "assignment1.wat")
!wat2wasm assignment1.wat
runpywasm("assignment1.wasm")

5.0
5.0
5.777777194976807


#####   Assign larger type to smaller type: lose precision
```
should output
5
5.6666665
5
```

In [38]:
compileString("""
var x: integer
var y: float
program p
    x := 5.66666f            //assign float to Int type variable
    y := 5.66666666666       //assign double to Float type variable    
    write(x)                 //5
    write(y)                 //5.6666665
    x := 5.66666666666      //assign Double to Int type variable 
    write(x)                 //5
""", "assignment2.wat")
!wat2wasm assignment2.wat
runpywasm("assignment2.wasm")

5
5.6666665
5


#####   Variables Assignement
```
should output 
8.0
8.0
9
9.5
```

In [39]:
compileString("""
var x: integer
var y: float
var z: double
program p
    x := 8
    y := x                  //assign interger value to float variable
    z := x                  //assign interger value to doule variable
    write(y)           //8.0
    write(z)          //8.0
    z := 9.5
    x := z                  //assign float type to interger variable
    y := z                  //assign float type to float variable
    write(x)                //9
    write(y)           //9.5
""", "assignent4.wat")
!wat2wasm assignent4.wat
runpywasm("assignent4.wasm")

8.0
8.0
9
9.5


#####   Binary operation for mixed types- the type of result is the same as the larger type of the operands
```
should output:
4.5
5.0
0.5
1.25
```

In [40]:
compileString("""
program binaryOpMixType
    var x: double
    var y: integer
    x := 2.5
    y := 2
    write(x + y)         //4.5
    write(x × y)         //5.0
    write(x - y)          //0.5
    write(x div y)        //1.25
""", "binaryOpMixType.wat")
!wat2wasm binaryOpMixType.wat
runpywasm("binaryOpMixType.wasm")

4.5
5.0
0.5
1.25


#####   Relation operations for mixed  double and float types
```
should output
0  (false)
1  (true)
0  (false)
1  (true)
0  (false)
1  (true)
```

In [41]:
compileString("""
program relationdMixDoubleFloat
    var x: double
    var y: float
      x := 2.1; y := 2.13f;
      if x = y then write(1) else write(0)
      if x ≠ y then write(1) else write(0)      
      if x > y then write(1) else write(0)
      if x < y then write(1) else write(0)      
      if x ≥ y then write(1) else write(0)
      if x ≤ y then write(1) else write(0)
""", "relationdMixDoubleFloat.wat")
!wat2wasm relationdMixDoubleFloat.wat
runpywasm("relationdMixDoubleFloat.wasm")

0
1
0
1
0
1


#####   Relation operations for mixed  double and int types
```
should output 
0 (false)
1 (true)
0 (false)
1 (true)
0 (false)
1 (true)
```

In [42]:
compileString("""
program relationdMixDoubleInt
    var x: double
    var y: integer
      x := 2.1; y := 3;
      if x = y then write(1) else write(0)
      if x ≠ y then write(1) else write(0)      
      if x > y then write(1) else write(0)
      if x < y then write(1) else write(0)      
      if x ≥ y then write(1) else write(0)
      if x ≤ y then write(1) else write(0)
""", "relationdMixDoubleInt.wat")
!wat2wasm relationdMixDoubleInt.wat
runpywasm("relationdMixDoubleInt.wasm")

0
1
0
1
0
1


#####   Relation operations for mixed  float and int types
```
should output
0  (false)
1  (true)
0  (false)
1  (true)
0  (false)
1  (true)
```

In [43]:
compileString("""
program relationdMixFloatInt
    var x: float
    var y: integer
      x := 2.1f; y := 3;
      if x = y then write(1) else write(0)
      if x ≠ y then write(1) else write(0)      
      if x > y then write(1) else write(0)
      if x < y then write(1) else write(0)      
      if x ≥ y then write(1) else write(0)
      if x ≤ y then write(1) else write(0)
""", "relationdMixFloatInt.wat")
!wat2wasm relationdMixFloatInt.wat
runpywasm("relationdMixFloatInt.wasm")

0
1
0
1
0
1


### 2.2 Explicit type conversion

#####   Type casting - convert to integer explicitly 
```
should output:
5
6
0
```

In [44]:
compileString("""
var x: integer
var y: float
program p    
    x := (int) 1.3 × 5           //5   convert 1.3 to interger 1 first and then multiply 5
    write(x)
    x := (int) (1.3 × 5)         //6    multiply 1.3 to 5 and then convert the result to integer
    write(x)
    x :=  (int) 2.5 div 5        //0    convert 2.5 to interger 2 first and then div 5
    write(x)
""", "typeCasting1.wat")
!wat2wasm typeCasting1.wat
runpywasm("typeCasting1.wasm")

5
6
0


#####   Type casting - convert interger to float or double explicitly
```
should output:
5.0
5.0
```

In [45]:
compileString("""
var x: float
var y: double
program p    
    x := (float) 1 × 5          //5.0
    write(x)
    y := (double) 1 × 5         //5.0
    write(y)
""", "typeCasting2.wat")
!wat2wasm typeCasting2.wat
runpywasm("typeCasting2.wasm")

5.0
5.0


<a name="link3"></a>
## 3 Unicode symbols

#####   SQRT should out put result for float and double types only 
```
should output:
4.0
4.0


```

In [46]:
compileString("""
var x: float
var y: double
var z: integer
program p              
    x := √16.0f
    write(x)
    y := √16.0
    write(y)
""", "sqrt.wat")
!wat2wasm sqrt.wat
runpywasm("sqrt.wasm")

4.0
4.0


#####   ABSVALUE should out put result for float and double types only 
```
should output:

16.0
8.8
16.0
6.1


```

In [47]:
compileString("""
var x: float
var y: double
var z: integer
program p              
    x := |(-16.0f)|
    write(x)
    x := |(2.1f-10.9f)|
    write(x)
    y := |(-16.0)|
    write(y)
    y := |(-9.1+3.0)|
    write(y)
""", "absv.wat")
!wat2wasm absv.wat
runpywasm("absv.wasm")

16.0
8.8
16.0
6.1


#####   CEiLING should out put result for float and double types only 
```
should output:

-16.0
9.0
-17.0
10.0


```

In [48]:
compileString("""
var x: float
var y: double
var z: integer
program p              
    x := ⎡(-16.1f)⎤
    write(x)
    x := ⎡(8.9f)⎤
    write(x)
    y := ⎡(-17.5)⎤
    write(y)
    y := ⎡(9.1)⎤
    write(y)
""", "ceil.wat")
!wat2wasm ceil.wat
runpywasm("ceil.wasm")

-16.0
9.0
-17.0
10.0


#####   FLOOR should out put result for float and double types only 
```
should output:

-17.0
8.0
-18.0
9.0


```

In [49]:
compileString("""
var x: float
var y: double
var z: integer
program p              
    x := ⎣(-16.1f)⎦
    write(x)
    x := ⎣(8.9f)⎦
    write(x)
    y := ⎣(-17.5)⎦
    write(y)
    y := ⎣(9.1)⎦
    write(y)
""", "floor.wat")
!wat2wasm floor.wat
runpywasm("floor.wasm")

-17.0
8.0
-18.0
9.0


<a name="link4"></a>
## 4 operator for max and min

#####   Max operator  
```
should output:
0.0
-1.0
2.33
2.3
2.33
2.3
300.198
300.198
300.198
300.198
2.33
```

In [50]:
compileString("""
var x: float
var y: double
var z: integer
var a: float
var b: double
var c: integer
program p              
    z := -1
    c := 0
    y := 2.3
    b := 2.33
    x := 300.198f
    a := -193.49772f
    write(z max c)
    write(z max a)
    write(z max b)
    write(y max a)
    write(y max b)
    write(y max c)
    write(x max a)
    write(x max b)
    write(x max c)
    write(x max y max z)
    write(a max b max c)
""", "max1.wat")
!wat2wasm max1.wat
runpywasm("max1.wasm")

0.0
-1.0
2.3299999999999996
2.3
2.3299999999999996
2.3
300.198
300.197998046875
300.198
300.197998046875
2.3299999999999996


#####   Max operator  
```
should output:
-1.0
-193.49772
-1.0
-193.49772
2.3
0.0
-193.49772
2.33
0.0
-1.0
-193.29772
```

In [51]:
compileString("""
var x: float
var y: double
var z: integer
var a: float
var b: double
var c: integer
program p              
    z := -1
    c := 0
    y := 2.3
    b := 2.33
    x := 300.198f
    a := -193.49772f
    write(z min c)
    write(z min a)
    write(z min b)
    write(y min a)
    write(y min b)
    write(y min c)
    write(x min a)
    write(x min b)
    write(x min c)
    write(x min y min z)
    write(a min b min c)
""", "min1.wat")
!wat2wasm min1.wat
runpywasm("min1.wasm")

-1.0
-193.49773
-1.0
-193.4977264404297
2.3
0.0
-193.49773
2.3299999999999996
0.0
-1.0
-193.4977264404297


#####   Mixed max and min
```
should output:
0.0
0.0
```

In [52]:
compileString("""
var x: float
var y: double
var z: integer
var a: float
var b: double
var c: integer
program p              
    z := -1
    c := 0
    y := 2.3
    b := 2.33
    x := 300.198f
    a := -193.49772f
    write(a min b max c)
    write(a max b min c)
""", "minAndMax.wat")
!wat2wasm minAndMax.wat
runpywasm("minAndMax.wasm")

0.0
0.0


<a name="link5"></a>
## 5 Quanfified expressions

#####   Sum and max of an array for the range of index - integer type - global variable
```
should output:
11
22
7
6
```

In [53]:
compileString("""
type T = [2..10] → integer
var a: T
program p
  var x, y: integer
    a[2] := 1
    a[3] := 7
    a[4] := 3
    a[5] := 4
    a[6] := 3
    a[7] := 6
    a[8] := 5
    a[9] := 4
    x := sum(a[2:5])         //1 + 7 + 3 = 11
    write(x)
    x := sum(a[5:10])         //4 + 3 + 6 + 5 + 4 = 22
    write(x)
    y := max(a[2:5])          //7
    write(y)
    y := max(a[5:10])          //6
    write(y)    
""", 'arraySumMax1.wat', target = 'wat')
!wat2wasm arraySumMax1.wat
runpywasm('arraySumMax1.wasm')

11
22
7
6


#####   Sum and max of an array for the range of index - float type - global variable
```
should output:
12.5
24.5
7.5
6.5
```

In [54]:
compileString("""
type T = [2..10] → float
var a: T
program p
  var x, y: float
    a[2] := 1.5f
    a[3] := 7.5f
    a[4] := 3.5f
    a[5] := 4.5f
    a[6] := 3.5f
    a[7] := 6.5f
    a[8] := 5.5f
    a[9] := 4.5f
    a[10] := 7.5f
    x := sum(a[2:5])         //1.5 + 7.5 + 3.5 = 12.5
    write(x)
    x := sum(a[5:10])         //4.5 + 3.5 + 6.5 + 5.5 + 4.5 = 24.5
    write(x)
    y := max(a[2:5])          //7.5
    write(y)
    y := max(a[5:10])          //6.5
    write(y) 
""", 'arraySumMax2.wat', target = 'wat')
!wat2wasm arraySumMax2.wat
runpywasm('arraySumMax2.wasm')

12.5
24.5
7.5
6.5


#####   Sum and max of an array for the range of index - double type - global variable
```
should output:
12.5
24.5
7.5
6.5
```

In [55]:
compileString("""
type T = [2..10] → double
var a: T
program p
  var x, y: double
    a[2] := 1.5
    a[3] := 7.5
    a[4] := 3.5
    a[5] := 4.5
    a[6] := 3.5
    a[7] := 6.5
    a[8] := 5.5
    a[9] := 4.5
    x := sum(a[2:5])         //1.5 + 7.5 + 3.5 = 12.5
    write(x)
    x := sum(a[5:10])         //4.5 + 3.5 + 6.5 + 5.5 + 4.5 = 24.5
    write(x)
    y := max(a[2:5])          //7.5
    write(y)
    y := max(a[5:10])          //6.5
    write(y) 
""", 'arraySumMax3.wat', target = 'wat')
!wat2wasm arraySumMax3.wat
runpywasm('arraySumMax3.wasm')

12.5
24.5
7.5
6.5


#####   Sum and max of an array for the range of index - integer type - local variable

```
should output:
11
22
7
6
```

In [56]:
compileString("""
program p
  type T = [2..10] → integer
  var a: T
  var x, y: integer
    a[2] := 1
    a[3] := 7
    a[4] := 3
    a[5] := 4
    a[6] := 3
    a[7] := 6
    a[8] := 5
    a[9] := 4
    x := sum(a[2:5])         //1 + 7 + 3 = 11
    write(x)
    x := sum(a[5:10])         //4 + 3 + 6 + 5 + 4 = 22
    write(x)
    y := max(a[2:5])          //7
    write(y)
    y := max(a[5:10])          //6
    write(y)    
""", 'arraySumMax4.wat', target = 'wat')
!wat2wasm arraySumMax4.wat
runpywasm('arraySumMax4.wasm')

11
22
7
6


#####   Sum and max of an array for the range of index - float type - local variable

```
should output:
12.5
24.5
7.5
6.5
```

In [57]:
compileString("""
program p
    type T = [2..10] → float
    var a: T
    var x, y: float
    a[2] := 1.5f
    a[3] := 7.5f
    a[4] := 3.5f
    a[5] := 4.5f
    a[6] := 3.5f
    a[7] := 6.5f
    a[8] := 5.5f
    a[9] := 4.5f
    x := sum(a[2:5])         //1.5 + 7.5 + 3.5 = 12.5
    write(x)
    x := sum(a[5:10])         //4.5 + 3.5 + 6.5 + 5.5 + 4.5 = 24.5
    write(x)
    y := max(a[2:5])          //7.5
    write(y)
    y := max(a[5:10])          //6.5
    write(y) 
""", 'arraySumMax5.wat', target = 'wat')
!wat2wasm arraySumMax5.wat
runpywasm('arraySumMax5.wasm')

12.5
24.5
7.5
6.5


#####   Sum and max of an array for the range of index - double type - local variable

```
should output:
12.5
24.5
7.5
6.5
```

In [58]:
compileString("""
program p
    type T = [2..10] → double
    var a: T
    var x, y: double
    a[2] := 1.5
    a[3] := 7.5
    a[4] := 3.5
    a[5] := 4.5
    a[6] := 3.5
    a[7] := 6.5
    a[8] := 5.5
    a[9] := 4.5
    x := sum(a[2:5])         //1.5 + 7.5 + 3.5 = 12.5
    write(x)
    x := sum(a[5:10])         //4.5 + 3.5 + 6.5 + 5.5 + 4.5 = 24.5
    write(x)
    y := max(a[2:5])          //7.5
    write(y)
    y := max(a[5:10])          //6.5
    write(y) 
""", 'arraySumMax6.wat', target = 'wat')
!wat2wasm arraySumMax6.wat
runpywasm('arraySumMax6.wasm')

12.5
24.5
7.5
6.5


<a name="link6"></a>
## 6 Programs

##### Procedures
`should output 9.0`

In [59]:
compileString("""
var g: double         // global variable
procedure q(v: double) // value parameter
  var l: double        // local variable
    l := 9.0
    if l > v then write(l) // writes 9
    else write(g)
program p
  g := 5.01; q(7.01)
""", 'proc.wat', target = 'wat')
!wat2wasm proc.wat
runpywasm('proc.wasm')

9.0


##### Double type as parameters of procedures
`should output 7.02 and -1`

In [60]:
compileString("""
procedure pro2(x, y: double) → (q, r: double)
    q, r := x + y, x - y

program arithmetic
    var x, y, q, r: double
      x := 3.01; y := 4.01;
      q, r ← pro2(x, y);
      write(q); write(r)
""", "arith.wat")
!wat2wasm arith.wat
runwasm('arith.wasm')

<IPython.core.display.Javascript object>

##### Double type as parameters of procedures
```
should output:
3.1
7.1
5.1
```

In [61]:
compileString("""
type T = [3..9] → double
var x: T
procedure r(a: T)
  write(a[4]); a[8] := 7.1       // writes 3.1
procedure q(a: T)
  r(a); write(a[8]); a[6] := 5.1 // writes 7.1
program p
  x[4] := 3.1; q(x); write(x[6]) // writes 5.1
""", 'arraypara.wat', target = 'wat')
!wat2wasm arraypara.wat
runpywasm('arraypara.wasm')

3.1
7.1
5.1


##### Array copy
```
should output:
3.1
5.1
0
```

In [62]:
compileString("""
var c: [0 .. 1] → double
var a, b: [2 .. 9] → double
program p
  b[2] := 3.1; b[3] := 5.1
  a := b
  write(a[2]); write(a[3]); write(a[4]) // writes 3.1, 5.1, 0
""", 'arraycopy.wat', target = 'wat')
!wat2wasm --enable-bulk-memory arraycopy.wat
runwasm('arraycopy.wasm')

<IPython.core.display.Javascript object>

##### 2D array
```
should output:
0
3.1
5.1
```

In [63]:
compileString("""
type A = [2 .. 9] → double
type B = [0 .. 1] → A
var b: B
procedure q(x: A) → (y: A)
    y := x; write(x[4]) // writes 0
program p
  b[1][2] := 3.1; b[1][3] := 5.1
  b[0] ← q(b[1])
  write(b[0][2]); write(b[0][3]) // writes 3.0, 5.0
""", 'arrayvalueresult.wat', target = 'wat')
!wat2wasm --enable-bulk-memory arrayvalueresult.wat
runwasm('arrayvalueresult.wasm')

<IPython.core.display.Javascript object>

##### local  array
```
should output:
3.1
5.1
0
```

In [64]:
compileString("""
type A = [2 .. 9] → double
type B = [0 .. 1] → A
procedure q(x: A)
  var y: A
    y := x
    write(x[2]); write(x[3]); write(x[4]) // writes 3.1, 5.1, 0
program p
  var b: B
    b[1][2] := 3.1; b[1][3] := 5.1
    q(b[1])
""", 'localarray.wat', target = 'wat')
!wat2wasm --enable-bulk-memory localarray.wat
runwasm('localarray.wasm')

<IPython.core.display.Javascript object>